In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
!pip install -q -U git+https://github.com/sbrugman/SDGym.git@v0.2.2-hw

In [ ]:
!pip install -q -U ../../

In [2]:
from timeit import default_timer as timer
from functools import partial
from random import choices
import logging

In [3]:
import sdgym
from sdgym import load_dataset
from sdgym import benchmark
from sdgym import load_dataset

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx

In [5]:
import pgmpy

from pgmpy.models import BayesianModel
from pgmpy.estimators import TreeSearch, HillClimbSearch, BicScore, ExhaustiveSearch, BayesianEstimator
from pgmpy.sampling import BayesianModelSampling

In [6]:
import xgboost as xgb
from xgboost import XGBClassifier

In [7]:
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.isotonic import IsotonicRegression

In [8]:
from scipy import interpolate

In [9]:
from synthsonic.models.kde_utils import kde_smooth_peaks_1dim, kde_smooth_peaks
from synthsonic.models.kde_copula_nn_pdf import KDECopulaNNPdf

In [13]:
import matplotlib.pyplot as plt
%matplotlib inline

In [14]:
logging.basicConfig(level=logging.INFO)

In [15]:
dataset_name = 'census_numeric'

In [16]:
data, categorical_columns, ordinal_columns = load_dataset(dataset_name)

INFO:sdgym.data:Loading dataset census (numeric variables, -1 distinct values threshold)
INFO:sdgym.data:Columns to zero-code: []


In [17]:
data.shape

(199523, 8)

In [18]:
categorical_columns, ordinal_columns

([], [])

In [19]:
for i in range(data.shape[1]):
    print (i, len(np.unique(data[:, i])))

0 91
1 1240
2 132
3 113
4 1478
5 7
6 53
7 2


# run sdgym

In [20]:
def KDECopulaNNPdf_Synthesizer(real_data, categorical_columns, ordinal_columns):
    assert len(categorical_columns + ordinal_columns) <= 1

    all_features = list(range(real_data.shape[1]))
    numerical_features = list(set(all_features) - set(categorical_columns + ordinal_columns))
    data = np.float64(real_data)
    
    n_samples = data.shape[0]
    n_features = data.shape[1]
    
    clf = xgb.XGBClassifier(
        n_estimators=250,
        reg_lambda=1,
        gamma=0,
        max_depth=9
    )
#     clf = MLPClassifier(alpha=0.1, random_state=0, max_iter=1000, early_stopping=True)
    
    kde = KDECopulaNNPdf(
        use_KDE=False, 
        categorical_columns=categorical_columns+ordinal_columns+[7],
        distinct_threshold=-1,
        n_bins=25,
#        clf=clf,
#         ordering='mi',
    )
    kde = kde.fit(data)
    
#     X_gen, sample_weight = kde.sample(n_samples)
    X_gen = kde.sample_no_weights(n_samples, show_progress=True, mode='cheap')
    
    X_gen[:, categorical_columns+ordinal_columns] = np.round(X_gen[:, categorical_columns+ordinal_columns])
    X_gen = np.float32(X_gen)
    
    print(X_gen[:, :5])

    return X_gen

In [21]:
def KDECopulaNNPdf_SynthesizerInteger(real_data, categorical_columns, ordinal_columns):
    """Census has integer only..."""
    data = KDECopulaNNPdf_Synthesizer(real_data, categorical_columns, ordinal_columns)
    data = np.round(data)
    
    print(data[:, :5])

    return data

In [23]:
from sdgym.synthesizers import (
    CLBNSynthesizer, CTGANSynthesizer, IdentitySynthesizer, IndependentSynthesizer,
    MedganSynthesizer, PrivBNSynthesizer, TableganSynthesizer, TVAESynthesizer,
    UniformSynthesizer, VEEGANSynthesizer)

all_synthesizers = [
    IdentitySynthesizer,
    #IndependentSynthesizer,
    #KDECopulaNNPdf_Synthesizer,
    KDECopulaNNPdf_SynthesizerInteger,
]

In [24]:
scores = sdgym.run(synthesizers=all_synthesizers, datasets=[dataset_name], iterations=1)

INFO:sdgym.benchmark:0%|          | 0/2 [00:00<?, ?it/s]
INFO:sdgym.benchmark:Evaluating IdentitySynthesizer on dataset census_numeric; iteration 0; 378.26 MB
INFO:sdgym.data:Loading dataset census (numeric variables, -1 distinct values threshold)
INFO:sdgym.data:Columns to zero-code: []
INFO:sdgym.benchmark:Running IdentitySynthesizer on dataset census_numeric; iteration 0; 384.78 MB
INFO:sdgym.synthesizers.base:Fitting IdentitySynthesizer
INFO:sdgym.synthesizers.base:Sampling IdentitySynthesizer
INFO:sdgym.benchmark:Scoring IdentitySynthesizer on dataset census_numeric; iteration 0; 407.24 MB
INFO:sdgym.evaluate:Evaluating using binary classifier DecisionTreeClassifier
INFO:sdgym.evaluate:Evaluating using binary classifier AdaBoostClassifier
INFO:sdgym.evaluate:Evaluating using binary classifier LogisticRegression
INFO:sdgym.evaluate:Evaluating using binary classifier MLPClassifier
/Users/maxbaak/opt/miniconda3/envs/synthsonic/lib/python3.8/site-packages/sklearn/neural_network/_multi

[[ 7.5000572e+01 -2.5121372e-03 -8.3705926e-01  1.1386655e-01
  -2.0990770e-01]
 [ 5.6999317e+01  4.7301441e-02 -4.6748024e-01 -3.7852775e-03
   1.7697155e+00]
 [ 7.4000633e+01 -4.6460390e-02  6.8876755e-01 -2.4327751e-02
   6.3903004e-01]
 ...
 [ 3.1000217e+01  1.8018819e-02 -1.3762525e+00  3.5974734e-02
   1.3139505e+00]
 [ 5.0998985e+01 -1.7354605e-01 -3.8619697e-02  1.1048457e-02
  -2.0061757e-01]
 [ 6.0006518e+00  1.5437769e-02  7.3588210e-01 -1.2666142e-01
  -6.8702638e-01]]
[[75. -0. -1.  0. -0.]
 [57.  0. -0. -0.  2.]
 [74. -0.  1. -0.  1.]
 ...
 [31.  0. -1.  0.  1.]
 [51. -0. -0.  0. -0.]
 [ 6.  0.  1. -0. -1.]]


INFO:sdgym.evaluate:Evaluating using binary classifier DecisionTreeClassifier
INFO:sdgym.evaluate:Evaluating using binary classifier AdaBoostClassifier
INFO:sdgym.evaluate:Evaluating using binary classifier LogisticRegression
INFO:sdgym.evaluate:Evaluating using binary classifier MLPClassifier
/Users/maxbaak/opt/miniconda3/envs/synthsonic/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (50) reached and the optimization hasn't converged yet.
  warnings.warn(
INFO:sdgym.benchmark:Finished KDECopulaNNPdf_SynthesizerInteger on dataset census_numeric; iteration 0; 405.17 MB
INFO:sdgym.benchmark:100%|##########| 2/2 [01:27<00:00, 43.89s/it]


In [27]:
scores

,census_numeric/accuracy,census_numeric/f1,timestamp
VEEGANSynthesizer,NaN,NaN,2020-04-12 09:41:35.096775
CLBNSynthesizer,NaN,NaN,2020-10-17 09:46:54.494331
CTGAN,NaN,NaN,2020-10-17 09:46:54.494331
CTGANSynthesizer,NaN,NaN,2020-10-17 09:46:54.494331
CopulaGAN,NaN,NaN,2020-10-17 09:46:54.494331
GaussianCopulaCategorical,NaN,NaN,2020-10-17 09:46:54.494331
GaussianCopulaCategoricalFuzzy,NaN,NaN,2020-10-17 09:46:54.494331
GaussianCopulaOneHot,NaN,NaN,2020-10-17 09:46:54.494331
IndependentSynthesizer,NaN,NaN,2020-10-17 09:46:54.494331
MedganSynthesizer,NaN,NaN,2020-10-17 09:46:54.494331


In [28]:
scores.tail(4)

,census_numeric/accuracy,census_numeric/f1,timestamp
TableganSynthesizer,NaN,NaN,2020-10-17 09:46:54.494331
UniformSynthesizer,NaN,NaN,2020-10-17 09:46:54.494331
IdentitySynthesizer,0.867410,0.346562,2021-05-13 20:44:48.081361
KDECopulaNNPdf_SynthesizerInteger,0.860925,0.251266,2021-05-13 20:44:48.081361
